# TFIDF

### Libraries

In [1]:
!pip install youtokentome
!pip install hazm
! pip install gdown
# fasttext
!git clone https://github.com/facebookresearch/fastText.git 
!cd fastText && pip install .

In [31]:
import os
import numpy as np
import codecs
import hazm
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import youtokentome as yttm
import fasttext.util
import fasttext

### Get The Skip-gram Fasttext Model

In [32]:
SKIPGRAM_MODEL_FILE_ID = '1wPnMG9_GNUVdSgbznQziQc5nMWI3QKNz'

In [2]:
# Let's explore the SKIPGRAM Model

!gdown --id $SKIPGRAM_MODEL_FILE_ID 

In [34]:
# loading the Model
model_skipgram = fasttext.load_model('farsi-dedup-skipgram.bin')

# official persian fasttext
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
# !gunzip cc.fa.300.bin.gz
# model_skipgram = fasttext.load_model('cc.fa.300.bin')

### Download Data

In [3]:
! wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P ./data
! wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip -P ./data
! unzip ./data/doc_collection.zip -d ./data
! wget https://raw.githubusercontent.com/SajjadMb/Masnavi_NLP_Toolkit/main/stopwords.txt -P ./data

### Read data

set path of data in `PATH` variable

In [36]:
PATH = './data/IR_dataset/'
PATH = PATH.rstrip('/')

store txt files into a list named `doc`

In [58]:
docs = []
for index in range(0, 3258):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
        docs.append(file_reader.read())

show example of data

In [5]:
for doc in docs[:3]:
    print (doc[:500])
    print('-' * 50)

### data clean

In [56]:
stopwords = [x.strip() for x in codecs.open('./data/stopwords.txt','r','utf-8').readlines()]
normalizer = hazm.Normalizer()

def remove_spaces_reg():
    # return space patterns
    space_pattern = r"[\xad\ufeff\u200e\u200d\u200b\x7f\u202a\u2003\xa0\u206e\u200c\x9d]"
    return re.compile(space_pattern)

def remove_symbols_reg():
    # return symbol patterns
    symbol_patterns = r"(\d|\"|'ٍ|¬|[؛“،,”‘۔’’‘–]|[|\.÷+\]\[\)\(\:\-\?»\=\{}\*«»_…\؟!/ـ]|[۰'ٓ۫'ٔ]|[ٓٔ]|[ًٌٍْﹼ،َُِّ«ٰ»ٖء])"
    return re.compile(symbol_patterns)

def remeove_arabic(text):
    # remove arabic alphabet
    mapping = {
        u"ۀ" : u"ه",
        u"ة" : u"ت",
        u"ي" : u"ی",
        u"ؤ" : u"و",
        u"إ" : u"ا",
        u"ٹ" : u"ت",
        u"ڈ" : u"د",
        u"ئ" : u"ی",
        u"ﻨ" : u"ن",
        u"ﺠ" : u"ج",
        u"ﻣ" : u"م",
        u"ﷲ" : u"",
        u"ﻳ" : u"ی",
        u"ٻ" : u"ب",
        u"ٱ" : u"ا",
        u"ڵ" : u"ل",
        u"ﭘ" : u"پ",
        u"ﻪ" : u"ه",
        u"ﻳ" : u"ی",
        u"ٻ" : u"ب",
        u"ں" : u"ن",
        u"ٶ" : u"و",
        u"ٲ" : u"ا",
        u"ہ" : u"ه",
        u"ﻩ" : u"ه",
        u"ﻩ" : u"ه",
        u"ك" : u"ک",
        u"ﺆ" : u"و",
        u"أ" : u"ا",
        u"ﺪ" : u"د"
    }

    arabic_keys =  re.compile(r"(" + "|".join(mapping.keys()) + r")")
    return arabic_keys.sub(lambda x: mapping[x.group()], text)


spaces_reg = remove_spaces_reg() 
symbols_reg = remove_symbols_reg()


def clean_text(text, allspace=True, punc=True, stopw=True):
    if allspace:
        text = re.sub(spaces_reg, " ", text)
    if punc:
        text = re.sub(symbols_reg, "", text)

    text = remeove_arabic(text)
    text = normalizer.normalize(text)
    
    if stopw:
        regex = r"\b(?:" + "|".join(map(re.escape, stopwords)) + r")\b"
        text = re.sub(regex, " ", text)
        
    text = re.sub("(\s)+", " ", text)
    text = text.strip()
    return text

In [60]:
# cleaned data
docs = [clean_text(doc, stopw=False) for doc in docs]
for doc in docs[:3]:
    print (doc[:500])
    print('-' * 50)

برخی از هواداران مصدق یا اعضای جبهه ملی که در زمان نخست وزیری مصدق از جبهه ملی یا از هییت وزیران کنار گذاشته شده یا کنار رفتند پس از جدایی از مصدق به انتقاد از کارنامه وی پرداختند و حتی برای سرنگونی اش تلاش کردند برخی از این افراد عبارت اند از فضل الله زاهدی نخست وزیر کودتا علی امینی حسین مکی که در آغاز سرباز فداکار وطن نامیده شد ولی در پایان به دلیل مخالفت با مصدق از سوی هواداران جبهه ملی سرباز خطاکار وطن خطاب می‌شد مظفر بقایی به دلیل اتهام مشارکت در قتل سرتیپ افشار طوس و سپس اتهام شرکت در کودت
--------------------------------------------------
جبهه ملی ایران که به اختصار جبهه ملی نیز خوانده می‌شود سازمان سیاسی ملی گرای سکولار دموکرات و جمهوری خواه فعال در ایران است که در حال حاضر تحت رهبری دکتر موسویان فعالیت می‌کند جبهه ملی ایران در سال توسط سیاستمدارانی از قبیل محمد مصدق حسین فاطمی و کریم سنجابی تاسیس شد و به پیش نهاد حسین فاطمی ملی شدن صنعت نفت ایران را مطرح کرد دکتر مصدق در کتاب خاطرات خود می‌گوید پیشنهاد ملی شدن صنعت نفت در سراسر کشور ابتکار شادروان دکتر حسین فاطمی بود با پیشنها

### bpe

In [61]:
BPE_PATH = './bpe/'
BPE_PATH = BPE_PATH.rstrip('/')
os.makedirs(BPE_PATH, exist_ok=True)
train_data_path = BPE_PATH + "/train_data.txt"
model_path = BPE_PATH + "/bpe.model"

- Train

In [62]:
# Generating corpus file with training data
pretrained_bpe_texts = ' '.join(docs)

with open(train_data_path, "w") as w_file:
    w_file.write(pretrained_bpe_texts)

# Training model
yttm.BPE.train(data=train_data_path, vocab_size=5000, model=model_path, unk_id=1, bos_id=2, eos_id=3)

- Load
- Test

In [4]:
# Loading model
bpe = yttm.BPE(model=model_path)

# first document as a test 
test_text = docs[0]

# Two types of tokenization
print(bpe.encode(test_text, output_type=yttm.OutputType.SUBWORD))

### TFIDF TOP K Relevent Documents 

In [44]:
## function: tokenizer persian 

In [45]:
## list: stop word persian

In [64]:
class tfidf:
    
    def __init__(self, docs, ngram_range, bpe_model=None):
        self._min_df = 1
        self._max_df=0.5
        self._max_features=10000
        self._docs = docs
        self._bpe = bpe_model
        self._ngram_range = ngram_range
        if self._bpe:
            self._model_tfidf = TfidfVectorizer(analyzer="word", min_df=self._min_df, max_df=self._max_df, max_features=self._max_features, ngram_range=self._ngram_range, tokenizer= lambda x: self.bpe_tokenizer(x))
        else:
            self._model_tfidf = TfidfVectorizer(analyzer="word", min_df=self._min_df, max_df=self._max_df, max_features=self._max_features, ngram_range=self._ngram_range)

        self._matrix = self._model_tfidf.fit_transform(docs)
        self._feature_names = self._model_tfidf.get_feature_names_out()
        #Load skip-gram embedding vector for each TF-IDF term
        self._tfidf_emb_vecs = np.vstack([model_skipgram.get_word_vector(word) for word in self._feature_names])

        #To get a TF-IDF weighted skip-gram vector summary of each document, 
        #we just need to matrix multiply docs_vecs with tfidf_emb_vecs
        self._docs_emb = np.dot(self._matrix.toarray(), self._tfidf_emb_vecs)


    def bpe_tokenizer(self, text):
        tokens = self._bpe.encode(text, output_type=yttm.OutputType.SUBWORD)
        return tokens 
        

    def tfidf_top_k(self, query, k=2):
        
        query_tfidf = self._model_tfidf.transform([query])

        # Stores cosine similarity scores
        doc_scores = []

        # Compute the cosine similarity scores
        for doc in self._matrix:
            doc_scores.append(cosine_similarity(query_tfidf, doc)[0][0])

        # Sort list of doc_scores and return the top k indices of highest scores
        sorted_scores = sorted(enumerate(doc_scores), key=lambda ind_score: ind_score[1], reverse=True)
        top_doc_indices = [ind for ind, score in sorted_scores[:k]]

        return top_doc_indices



    def tfidf_weighted_top_k(self, query, k=2):

        query_tfidf = self._model_tfidf.transform([query])
        query_emb = np.dot(query_tfidf.toarray(), self._tfidf_emb_vecs)
        # Stores cosine similarity scores
        doc_scores = []

        # Compute the cosine similarity scores
        for doc in self._docs_emb:
            doc_scores.append(cosine_similarity(query_emb, doc.reshape(1, -1))[0][0])

        # Sort list of doc_scores and return the top k indices of highest scores
        sorted_scores = sorted(enumerate(doc_scores), key=lambda ind_score: ind_score[1], reverse=True)
        top_doc_indices = [ind for ind, score in sorted_scores[:k]]
      
        return top_doc_indices

In [47]:
query = "آدولف هیتلر شکست و مرگ"

###  Word Level: ngram (1,1)

In [48]:
tf_obj = tfidf(docs=docs, ngram_range=(1,1), bpe_model=None)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 349,
 345,
 343,
 344,
 466,
 347,
 481,
 341,
 463,
 1544,
 342,
 469,
 443,
 354,
 483,
 220,
 442,
 449]

###  Word Level: ngram (1,2)

In [49]:
tf_obj = tfidf(docs=docs, ngram_range=(1,2), bpe_model=None)
tf_obj.tfidf_top_k(query, 20)

[484,
 349,
 482,
 347,
 343,
 344,
 466,
 442,
 345,
 481,
 342,
 341,
 463,
 354,
 220,
 469,
 1544,
 443,
 483,
 449]

### BPE: ngram (1,1)

In [50]:
tf_obj = tfidf(docs=docs, ngram_range=(1,1), bpe_model=bpe)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 349,
 347,
 345,
 343,
 344,
 466,
 341,
 481,
 342,
 463,
 442,
 1544,
 220,
 483,
 443,
 469,
 2080,
 449]

### BPE: ngram (1,2)

In [51]:
tf_obj = tfidf(docs=docs, ngram_range=(1,2), bpe_model=bpe)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 349,
 442,
 347,
 344,
 345,
 466,
 343,
 341,
 1544,
 481,
 342,
 463,
 443,
 469,
 483,
 220,
 438,
 449]

TFIDF Weighted

In [65]:
tf_obj = tfidf(docs=docs, ngram_range=(1,2), bpe_model=None)
tf_obj.tfidf_weighted_top_k(query, 20)

[341,
 466,
 443,
 484,
 481,
 1544,
 345,
 343,
 349,
 482,
 442,
 347,
 438,
 344,
 342,
 351,
 348,
 2727,
 354,
 465]